# Data Exploration
## Anscombe's Quartet
The following code is adapted from https://matplotlib.org/stable/gallery/specialty_plots/anscombe.html.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split

In [ ]:
x = np.array([10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5])
y1 = np.array([8.04, 6.95, 7.58, 8.81, 8.33, 9.96, 7.24, 4.26, 10.84, 4.82, 5.68])
y2 = np.array([9.14, 8.14, 8.74, 8.77, 9.26, 8.10, 6.13, 3.10, 9.13, 7.26, 4.74])
y3 = np.array([7.46, 6.77, 12.74, 7.11, 7.81, 8.84, 6.08, 5.39, 8.15, 6.42, 5.73])
x4 = np.array([8, 8, 8, 8, 8, 8, 8, 19, 8, 8, 8])
y4 = np.array([6.58, 5.76, 7.71, 8.84, 8.47, 7.04, 5.25, 12.50, 5.56, 7.91, 6.89])

datasets = {
    'I': (x, y1),
    'II': (x, y2),
    'III': (x, y3),
    'IV': (x4, y4)
}

In [ ]:
# look at mean, stdev, corr coeff
for label, (x,y) in datasets.items():
    print(f"Dataset {label}:")
    print(f"  Mean:  {np.mean(y):.2f}")
    print(f"  Stdev: {np.std(y):.2f}")
    print(f"  R^2: {np.corrcoef(x,y)[0][1]**2:.2f}")
    # print(f"  Min: {np.min(y):.2f}")
    # print(f"  Max: {np.max(y):.2f}")

In [ ]:

fig, axs = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(6, 6),
                        gridspec_kw={'wspace': 0.08, 'hspace': 0.08})
axs[0, 0].set(xlim=(0, 20), ylim=(2, 14))
axs[0, 0].set(xticks=(0, 10, 20), yticks=(4, 8, 12))

for ax, (label, (x, y)) in zip(axs.flat, datasets.items()):
    ax.text(0.1, 0.9, label, fontsize=20, transform=ax.transAxes, va='top')
    ax.tick_params(direction='in', top=True, right=True)
    ax.plot(x, y, 'o')

    # linear regression
    p1, p0 = np.polyfit(x, y, deg=1)  # slope, intercept
    ax.axline(xy1=(0, p0), slope=p1, color='r', lw=2)

    # add text box for the statistics
    stats = (f'$\\mu$ = {np.mean(y):.2f}\n'
             f'$\\sigma$ = {np.std(y):.2f}\n'
             f'$r$ = {np.corrcoef(x, y)[0][1]:.2f}')
    bbox = dict(boxstyle='round', fc='blanchedalmond', ec='orange', alpha=0.5)
    ax.text(0.95, 0.07, stats, fontsize=9, bbox=bbox,
            transform=ax.transAxes, horizontalalignment='right')

plt.show()

## Datasaurus Dozen
From https://www.research.autodesk.com/publications/same-stats-different-graphs/

In [ ]:
# load the dataset
dsd = pd.read_csv("DatasaurusDozen.tsv", sep="\t")
dsd.head()

In [ ]:
dsd.info()

In [ ]:
dsd["dataset"].value_counts()

In [ ]:
fig, axs = plt.subplots(3, 4, sharex=True, sharey=True, figsize=(10,8))

for ax, name in zip(axs.flat, dsd["dataset"].unique()):
    slice = dsd.query("dataset == @name")
    ax.scatter(slice["x"], slice["y"])
    ax.set_title(name)

    print(f"Dataset {name}:")
    print(f"  Mean:  {np.mean(slice['y']):.2f}")
    print(f"  Stdev: {np.std(slice['y']):.2f}")
    print(f"  R^2: {np.corrcoef(slice['x'],slice['y'])[0][1]**2:.2f}")

# A more useful example
Let's use Kaggle's [Titanic dataset](https://www.kaggle.com/competitions/titanic/data). Kaggle has already split the data into train/test, so we'll just load the training data and pretend that test doesn't exist.

Many public datasets are pre-split. This is so that you can replicate published results exactly. In other cases (like Kaggle competitions), a final test set is kept secret, ensuring things really can't leak.

In [ ]:
td = pd.read_csv("titanic_train.csv")

In [ ]:
# info, describe, head
td.info()
td.describe()

In [ ]:
# Do more women survive than men? "Women and children first"
td[["Sex", "Survived"]].groupby("Sex").value_counts()

Yes, looks like sex is related to survival.

In [ ]:
# What about the relationship between age and survival?
ax = td[td["Survived"] == 0].hist(by="Sex", column="Age", figsize=(8,4), alpha=0.3, label="Did not survive")
td[td["Survived"] == 1].hist(ax=ax, by="Sex", column="Age", figsize=(8,4), alpha=0.3, label="Survived")
plt.legend()
plt.xlabel("Age")
plt.ylabel("# of people")

## Splitting the data

In [ ]:
# Splitting the data - random sample
td_train, td_test = train_test_split(td, test_size=0.2, random_state=1234)

In [ ]:
print(td_train["Survived"].value_counts() / len(td_train))
print(td_test["Survived"].value_counts() / len(td_test))

In [ ]:
# Splitting the data - stratified sample

In [ ]:
# Splitting the data - hashing to prevent changes
from zlib import crc32

# Passenger ID should be a unique identifier, but let's double check
print("Unique identifier:", td["PassengerId"].is_unique)

h_train_ids = td["PassengerId"].apply(lambda n: crc32(str(n).encode())) < 0.8 * 2**32
h_test_ids = np.ones_like(td["PassengerId"].values, dtype=bool)
h_test_ids[h_train_ids] = 0

print(f"Training fraction: {h_train_ids.sum() / len(td)}")
print(f"Testing fraction: {h_test_ids.sum() / len(td)}")

In [ ]:
# scatter plots
td.plot.scatter(x="Age", y="Fare", c="Survived", cmap="viridis")

In [ ]:
# maybe a log scale would help?
ax = td.plot.scatter(x="Age", y="Fare", c="Survived", cmap="viridis")
ax.set_yscale("log")

In [ ]:
# box plots
td.plot.box(by="Survived", layout=(2,3), figsize=(10,8))

## Extra plots
Small examples that don't really fit with the main flow.

A brief example of sampling bias

In [ ]:
from scipy.stats import binom
import matplotlib.pyplot as plt

p = 0.8 # ratio of likes cilantro to dislikes cilantro
buffer = 0.05 # plus/minus 5%
sample_sizes = [10, 100, 200, 500, 1000, 10000]
prob_bias = []

for n in sample_sizes:
    too_small = n * (p - buffer)
    too_large = n * (p + buffer)
    proba_too_small = binom(n, p).cdf(too_small - 1)
    proba_too_large = 1 - binom(n, p).cdf(too_large)
    prob_bias.append((proba_too_small + proba_too_large) * 100)

plt.plot(sample_sizes, prob_bias, "o-")
plt.xlabel("Sample size")
plt.ylabel("Probability of random sampling bias (%)")
plt.xscale("log")
plt.show()

for n, b in zip(sample_sizes, prob_bias):
    print(f"Sample size {n}: Bias = {b}")

In [ ]:
rng = np.random.default_rng()

ids = np.arange(10)
train = rng.choice(ids, 8, replace=False)
test = np.delete(ids, train)

print("Train:", train)
print("Test: ", test)

plt.bar(train, np.ones_like(train), label="train")
plt.bar(test, np.ones_like(test), label="test")
plt.ylim([0,1.2])
plt.legend()

In [ ]:
# deterministic approach: hashing
from zlib import crc32

# 80% of the maximum possible 32-bit hash value
test_thresh = 0.8 * 2**32

hash_vals = np.array([crc32(id) for id in ids])
train = ids[hash_vals < test_thresh]
test = np.delete(ids, train)

print("Train:", train)
print("Test: ", test)

plt.bar(train, np.ones_like(train), label="train")
plt.bar(test, np.ones_like(test), label="test")
plt.ylim([0,1.2])
plt.legend()